#Team Question -- Masketeers
Do the hours that restaurants and bars are open each week relate to the number of reviews they get per year?

What role does the size or location of the metro area play (do smaller cities have fewer reviews per restaurant)?

Does serving alcohol matter?

Does being open for certain meals matter?

# Analysis Approach

After careful consideration of different methods to analyze these questions our team decided to obtain the business hours for the restaurants and grouping them by their year of review and their year would be how to understand the impact the hours have on their reviews along with the yearly and weekly with regards to hours. In addition, we are going to determine the top 10 metro areas based on the number of business reviews and with that knowledge we will see how the restaurant location can influence the number of reviews a resturant receives. The restaurant's alcohol serving type should be found in order to be able to see how the number of reviews is affected by whether the restaurant serves alcohol or not. By getting the data for the meal times in which the restaurants are best fit to identify whether the meals in which a restaurant is open give insight into the number of reviews the restaurant will receive. We will be counting the reviews by the number of instances a restaurant appears in the Yelp review file.

### Data needed from Yelp:
1. Yelp business data
2. Yelp Reviews data
3. Yelp business categories data

### Sources used:
1. We will be using the files provided by the <a href="https://www.yelp.com/dataset" target="_blank">Yelp Dataset Challenge</a> this came in the form of a tar file that contained multiple files regarding Yelp data.
2. The files we will be using from the Yelp Dataset Challenge is the Business.bz2 file, categories.json, and reviews.bz2

#Importing data and software tools

To start, we will import the necessary software tools to help with the analysis and visualization.

In [0]:
from PIL import Image
TEMP_DIR = "/temp"

import base64

import pyspark.sql.functions as f

In [0]:
def getWidth(path):
  with Image.open(path) as img:
    width, height = img.size
    return(width)
  
def getDbfsPathName(path):
    # Get the fileinfo containing the path and name
  if path.startswith("/dbfs") != True:
    raise Exception("The path provided does not start with /dbfs")
  new_path = "dbfs:" + path[5:]
  # get the file info for the path
  file_list = dbutils.fs.ls(new_path)
  if len(file_list) != 1:
    raise Exception("The path provided is not a single file on dbfs")
  dbfs_path = file_list[0].path
  filename = file_list[0].name
  return(dbfs_path, filename)
  
def getTempPath(filename):
  # Create the temp directory if it does not exist
  temp_path = "file:" + TEMP_DIR
  dbutils.fs.mkdirs(temp_path)
  temp_list = dbutils.fs.ls(temp_path)
  # get a name to use for the copy
  temp_files = []
  for info in temp_list:
    temp_files.append(info.name)
  increment = 0
  new_name = filename
  while new_name in temp_files:
    increment+=1
    new_name = filename + "." + str(increment)
  access_path = TEMP_DIR + "/" + new_name # used for file opening
  return(access_path)

In [0]:
def showimage(path, width=0):
  image_string = ""
  img_tag = ""
  dbfs_path, filename = getDbfsPathName(path)
  access_path = getTempPath(filename)
  # copy the file
  copy_path = "file:" + access_path
  dbutils.fs.cp(dbfs_path,copy_path)
  with open(access_path, "rb") as image_file:
    image_string = base64.b64encode(image_file.read() ).decode('utf-8') 
    
  # Is the width setting a positive integer?  A width of 50 means 50%
  if width > 0 and width < 1:
    print("If the width parameter is specified, it must be 1 or more.  A width of 50 means 50%. The width entered was " + str(width) + ", so the original image width was used.")
    width = 0 #reset
    
  if width == 0:
    height = 0
    # Get the width and height of the image in pixels
    with Image.open(access_path) as img:
      width, height = img.size
      
    framewidth = width * 1.1
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  style="width:%ipx;height=%ipx;" /></div>''' % (framewidth,framewidth,image_string, width, height)
  else: # a width was specified
    originalWidth = getWidth(access_path)
    imagewidth = int( width / 100.0 * originalWidth)
    framewidth = int( imagewidth * 1.1 )
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  width="%ipx" height="auto"></div>''' % (framewidth,framewidth,image_string, imagewidth)
  # Clean up the file
  dbutils.fs.rm(copy_path)
  return(img_tag)

## Importing Business Data

First, we collect all of the information regarding the businesses from the business.bz2 file and store it into **`df_business`** This dataset will give us a quick glance 
of the bigger picture of the business attributes.  Once we have that information we can begin wrangling the data.

In [0]:
df_business = spark.read.json('/yelp/business.bz2')
print ("Total number of businesses:", df_business.count() )
df_business.printSchema()

Total number of businesses: 209393
root
-- address: string (nullable = true)
-- attributes: struct (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- BusinessParking: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: string (nullable = true)
 |-- HappyHour: string (nullable = true)
 |-- HasTV: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- NoiseLevel: string (nullable = true)
 |-- Open24Hours: string (nullable = true)
 |-- OutdoorSeating: string (nullable = true)
 |-- RestaurantsAttire: string (nullable = true)
 |-- RestaurantsCounterService: string (nullable = true)
 |-- RestaurantsDelivery: string (nullable = true)
 |-- RestaurantsGoodForGroups: string (nullable = true)
 |-- RestaurantsPriceRange2: string (nullable = true)
 |-- RestaurantsReservations: string (nullable = true)
 |-- RestaurantsTableService: string (nullable = true)
 |-- RestaurantsTakeOut: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- WheelchairAccessible: string (nullable = true)
 |-- WiFi: string (nullable = true)
-- business_id: string (nullable = true)
-- categories: string (nullable = true)
-- city: string (nullable = true)
-- hours: struct (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Thursday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
-- is_open: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- name: string (nullable = true)
-- postal_code: string (nullable = true)
-- review_count: long (nullable = true)
-- stars: double (nullable = true)
-- state: string (nullable = true)

## Identifying categories from the dataset
The code below is also importing data from the categories.json file. It helps us to identify what types of businesses are in the dataset based on the attributes linked to the business. Having identified the business attributes, we can narrow down list of businesses, selecting only the ones that are relevant to our question.

In [0]:
df_categories = spark.read.json("/yelp/categories.json")
print( "number of categories:", df_categories.count() )
df_categories.show()
df_categories.printSchema()

number of categories: 1565
+------------------+--------------------+-----------------+---------------+-------------------+
 alias| country_blacklist|country_whitelist| parents| title|
+------------------+--------------------+-----------------+---------------+-------------------+
 3dprinting| null| null|[localservices]| 3D Printing|
 abruzzese| null| [IT]| [italian]| Abruzzese|
 absinthebars| null| [CZ]| [bars]| Absinthe Bars|
 acaibowls|[TR, IT, CL, MX, ...| null| [food]| Acai Bowls|
 accessories| null| null| [fashion]| Accessories|
 accountants| null| null| [professional]| Accountants|
 acnetreatment| null| null| [beautysvc]| Acne Treatment|
 active| null| null| []| Active Life|
 acupuncture| null| null| [health]| Acupuncture|
 addictionmedicine| [JP]| null| [physicians]| Addiction Medicine|
 adoptionservices| null| null|[localservices]| Adoption Services|
 adult| null| null| [shopping]| Adult|
 adultedu| null| null| [education]| Adult Education|
adultentertainment| null| null| [nightlife]|Adult Entertainment|
 advertising| null| null| [professional]| Advertising|
 aerialfitness| null| null| [fitness]| Aerial Fitness|
 aerialtours| null| null| [tours]| Aerial Tours|
 aestheticians|[BE, FR, CL, MX, ...| null| [medicalspa]| Aestheticians|
 afghani| [MX, TR]| null| [restaurants]| Afghan|
 african| [TR]| null| [restaurants]| African|
+------------------+--------------------+-----------------+---------------+-------------------+
only showing top 20 rows

root
-- alias: string (nullable = true)
-- country_blacklist: array (nullable = true)
 |-- element: string (containsNull = true)
-- country_whitelist: array (nullable = true)
 |-- element: string (containsNull = true)
-- parents: array (nullable = true)
 |-- element: string (containsNull = true)
-- title: string (nullable = true)

##Working with Review data
Unlike the other datasets, the review data is a massive file and must be imported differently through a `select` method. In this method, `review_id` and `text` will be excluded because they are not relevant to the question and will take longer to import. We will also cache the dataframe to hold the data in memory.

In [0]:
if ( spark.catalog._jcatalog.tableExists("reviews_without_text_table") ):
  df_reviews = spark.read.format('parquet').table("reviews_without_text_table").cache()
  print("loaded review data from table")
else:
  df_reviews = spark.read.json("/yelp/review.bz2").select("business_id","user_id", "cool", "funny","useful","date","stars").cache()
  # Does not already exist as a table, so write it out
  df_reviews.write.mode("overwrite").format('parquet').saveAsTable("reviews_without_text_table")
  print("loaded review data from file")
  
print ("total number of review records: ", df_reviews.count() )
df_reviews.printSchema()
df_reviews.show(10, truncate=22)
df_reviews.unpersist()

df_reviews.createOrReplaceTempView("reviews")

loaded review data from table
total number of review records: 8021122
root
-- business_id: string (nullable = true)
-- user_id: string (nullable = true)
-- cool: long (nullable = true)
-- funny: long (nullable = true)
-- useful: long (nullable = true)
-- date: string (nullable = true)
-- stars: double (nullable = true)

+----------------------+----------------------+----+-----+------+-------------------+-----+
 business_id| user_id|cool|funny|useful| date|stars|
+----------------------+----------------------+----+-----+------+-------------------+-----+
3chd-CjwQz7Pap_MQEWhWg|c4Y3EX5aXYixkyVD6uSMag| 2| 1| 1|2019-09-04 19:16:55| 4.0|
XV45a3fz1d7SRoash2vu3g|fHTAUYCd5HuboQp6HymzjQ| 1| 1| 13|2016-03-01 21:46:00| 3.0|
BxcLVgA6BndKDJol0ZZu0g|ecXDhY91U-ORKk_U7wwdeA| 0| 0| 0|2019-11-18 07:13:06| 3.0|
nSPh4_BKX3Fch0TOahe-vQ|ZKpETYqLymDFL76hwkUaSQ| 0| 0| 3|2019-08-15 16:30:53| 1.0|
KrcmoxVBX9o3OxFogGx-RA|zcp389V72h0DKl69X72wKQ| 4| 3| 6|2019-10-24 20:37:02| 3.0|
ewn93D_Q6bnVosJtwF9oiQ|xbksPRSmtsmDNiR1VoTOPA| 0| 0| 0|2019-11-19 15:33:33| 2.0|
fKjeOlJTQ1W0cSDRYFdZmA|dG-J0IG122Nwqp0Co21Rkw| 0| 0| 1|2019-10-21 00:15:38| 1.0|
b7xistZfcbSO6_0Mg-heRA|O4ADr-eEYkP1OKQUxi49EQ| 0| 0| 13|2017-12-22 07:26:39| 1.0|
6pM2YxQH8dQYs07_CH3Z6g|LeSXKUJeFvX2aSvfPnl7dg| 0| 0| 2|2019-04-15 14:20:28| 5.0|
iVjeR9z1xVt2ZpxVU4UcXQ|_fX--5YqPXMB4b7q22VWkw| 0| 0| 0|2019-11-16 22:29:28| 5.0|
+----------------------+----------------------+----+-----+------+-------------------+-----+
only showing top 10 rows

#### Cleaning and Organizing Review Data
After importing the review data, we begin to organize it by sorting and grouping the reviews based on the `business_id` and `year`. This way, we can narrow down our information and see how many reviews a business may get within a year, rather than an aggregate of all the years.

In [0]:
df_reviews_processed = spark.sql("""
SELECT business_id, 
       YEAR(date) AS review_year, 
       COUNT(business_id) AS review_count
FROM reviews
group by business_id,Year(date)""")

print("Total number of review entries:", df_reviews_processed.count())
df_reviews_processed.show()

Total number of review entries: 1008217
+--------------------+-----------+------------+
 business_id|review_year|review_count|
+--------------------+-----------+------------+
P2dFr-Jg1OUoNMika...| 2019| 118|
z5eeJA-ItO9EZDkQU...| 2019| 25|
BjH8Xepc10i6OhCDQ...| 2018| 136|
lqpQE6hBAb9z-CvDn...| 2019| 8|
z56IwZfjgyEHnK67W...| 2015| 15|
eSILD9dtyDApgcL5L...| 2019| 14|
YGHbozrtjNu5Af-tz...| 2017| 3|
b1EPAu7qqlKqVeRw8...| 2019| 11|
5Q95IfYIbP6vXz4wj...| 2019| 47|
0gXYLVPNWz0WT8wXQ...| 2018| 122|
uKKNw68mZZaa1jcys...| 2019| 68|
UMHrFxT6S581dEZaR...| 2019| 3|
FpdIfwSZcoe673RPJ...| 2016| 2|
1OQy8OZUdQ5tRlGUG...| 2013| 3|
aqONNC5onqX6EqHHU...| 2013| 51|
yoJB1EQWfhGgTI48I...| 2015| 17|
vVzYDKCXaGhIRavkI...| 2019| 9|
ljLq8ZTLO52rzwsEe...| 2014| 22|
ZK3xWc5Ataig-zJ-F...| 2012| 1|
ffcjnok3HzZEE2rYH...| 2012| 1|
+--------------------+-----------+------------+
only showing top 20 rows

In [0]:
df_reviews_processed.createOrReplaceTempView("busReviews")

Since some of our data do not rely on yearly reports, we create a separate dataframe that groups the reviews based on `business_id`.

In [0]:
df_review_count = df_reviews_processed.select("business_id","review_year","review_count").groupBy("business_id").agg({"business_id": "count"}).\
                            withColumnRenamed("count(business_id)", "reviews")
print("total number of reviews:", df_review_count.count())
df_review_count.show()
df_review_count.createOrReplaceTempView("reviewcount")

total number of reviews: 209393
+--------------------+-------+
 business_id|reviews|
+--------------------+-------+
Npsws-Pn0S7vtCKdx...| 11|
avljb14OB8UkFTHVo...| 5|
nfSCAPBh1IoEznoJZ...| 11|
nCEomY3N-sYkoUhIr...| 12|
Dnq4w6ajMifZFFTIR...| 5|
o8ixVlZRwTlCG3Bnq...| 5|
fMfJ0LUG5iCoFJIgH...| 4|
cy9bWd0tufGp-xkA8...| 5|
LC5kF3rYFcsx7Mt7C...| 3|
yrpFPFl8qr1zIQnlC...| 3|
9Vf__OXZkR7pyp0Cb...| 7|
rBB5xNN0n5k29Pqq4...| 6|
RsREl3LkFGiwbt3kG...| 14|
l1SGwoX-qkd17XUNk...| 4|
cm2S29L_aAy6i3lj3...| 5|
-qL-wX-UuMaHDBA2X...| 8|
Zo3fwmOUaDpKvUhLN...| 11|
nJClfvVwSrr2uuo_y...| 9|
34NO2dIwpq4tbSxOs...| 2|
jY_HFOPDnluNESvlW...| 2|
+--------------------+-------+
only showing top 20 rows

## Selecting Business Data
For us to wrangle the data, we must profile it to get a better understanding of what we are working with. 

Here we create the dataframe `df_businessCat` which has all the businesses in the Yelp data, alongside their attributes and geographical information. 

Using the attributes, we can narrow down the businesses and only select the ones that are a restaurant or bar. The geographical data will help us determine their location so we can compare to see if location effects the number of reviews received for a business.

In [0]:
df_businessCat = df_business.select("business_id", "name", "state", "city" ,"attributes", "postal_code", "latitude", "longitude", "hours",f.split(df_business.categories,'\s*,\s*').alias("categories"))
df_businessCat.show(10, vertical = True, truncate = False)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | f9NumwFMBDn751xgFiRbNA 
 name | The Range At Lake Norman 
 state | NC 
 city | Cornelius 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, False,,,,,,,, False,,,,,,,,,,,,, 3,,,,,,] 
 postal_code | 28031 
 latitude | 35.4627242 
 longitude | -80.8526119 
 hours | [11:0-20:0, 10:0-18:0, 11:0-20:0, 13:0-18:0, 11:0-20:0, 11:0-20:0, 10:0-18:0] 
 categories | [Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping] 
-RECORD 1-----------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | Yzvjg0SayhoZgCljUJRF9Q 
 name | Carlos Santo, NMD 
 state | AZ 
 city | Scottsdale 
 attributes | [,,,,,,,,,,, True,,,,,,,, True,,,,,,,,,,,,,,,,,,,] 
 postal_code | 85258 
 latitude | 33.5694041 
 longitude | -111.8902637 
 hours | null 
 categories | [Health & Medical, Fitness & Instruction, Yoga, Active Life, Pilates] 
-RECORD 2-----------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | XNoUzKckATkOD1hP6vghZg 
 name | Felinus 
 state | QC 
 city | Montreal 
 attributes | null 
 postal_code | H4C 1P4 
 latitude | 45.479984 
 longitude | -73.58007 
 hours | null 
 categories | [Pets, Pet Services, Pet Groomers] 
-RECORD 3-----------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | 6OAZjbxqM5ol29BuHsil3w 
 name | Nevada House of Hose 
 state | NV 
 city | North Las Vegas 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}, False,,,,, True,,,,,,,,,,,,,,,, 4,,,,,,] 
 postal_code | 89030 
 latitude | 36.2197281 
 longitude | -115.1277255 
 hours | [7:0-16:0, 7:0-16:0,,, 7:0-16:0, 7:0-16:0, 7:0-16:0] 
 categories | [Hardware Stores, Home Services, Building Supplies, Home & Garden, Shopping] 
-RECORD 4-----------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | 51M2Kk903DFYI6gnB5I6SQ 
 name | USE MY GUY SERVICES LLC 
 state | AZ 
 city | Mesa 
 attributes | [,,,,,,,,, True,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 postal_code | 85205 
 latitude | 33.4280652 
 longitude | -111.7266485 
 hours | [9:0-16:0, 0:0-0:0,,, 9:0-16:0, 9:0-16:0, 9:0-16:0] 
 categories | [Home Services, Plumbing, Electricians, Handyman, Contractors] 
-RECORD 5-----------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | cKyLV5oWZJ2NudWgqs8VZw 
 name | Oasis Auto Center - Gilbert 
 state | AZ 
 city | Gilbert 
 attributes | [,,,,,,,,, True,,,,,,,,,,,,,,,,,,,,,,,,,,,,,] 
 postal_code | 85233 
 latitude | 33.3503993 
 longitude | -111.8271417 
 hours | [7:0-18:0, 7:0-18:0, 7:0-15:0,, 7:0-18:0, 7:0-18:0, 7:0-18:0] 
 categories | [Auto Repair, Automotive, Oil Change Stations, Transmission Repair] 
-RECORD 6-----------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | oiAlXZPIFm2nBCt0DHLu_Q 
 name | Green World Cleaners 
 state | NV 
 city | Las Vegas 
 attributes | [,,,,,,, True,, True, {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False},,,,,,,,,,,,,,,,,,,,,, 1,,,,,, u'no'] 
 postal_code | 89118 
 latitude | 36.0639767 
 longitude | -115.241463 
 hours | [7:0-19:0, 7:0-19:0, 9:0-17:0,, 7:0-19:0, 7:0-19:0, 7:0-19:0] 
 categories | [Dry Cleaning & Laundry, Local Services, Laundry Services] 
-RECORD 7--------

In [0]:
df_businessCat.createOrReplaceTempView("businessAll")

### Filtering the dataframe we just created to only get the relevant businesses
Using the temporary view `businessAll`, which has all the business information from `df_businessCat`, we can filter out the non-restaurant businesses from the dataset to create `df_RestnBars`. We do this by using `ARRAY_CONTAINS` to find the businesses with restaurant or bar in the categories, or alcohol listed in attributes. We take these extra steps to ensure that all the relevant businesses are selected. 

This is step is crucial for our question because we are only looking at restaurants and whether they serve alcohol, any other businesses will be irrelevant to our question.

In [0]:
df_RestnBars = spark.sql("""
SELECT *
FROM businessAll
WHERE ARRAY_CONTAINS(categories, "Restaurants") OR attributes.Alcohol IS NOT null OR ARRAY_CONTAINS(categories, "Bars")
""")
print("total number business: ", df_RestnBars.count())
df_RestnBars.show(truncate = False, vertical = True)


total number business: 68985
-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | pQeaRpvuhoEqudo3uymHIQ 
 name | The Empanadas House 
 state | IL 
 city | Champaign 
 attributes | [,, 'none',,,,, False,, True, None, False,,,,,,,, False,,,, True,, u'quiet',, False, u'casual',, True, False, 1, False,, True,,,] 
 postal_code | 61820 
 latitude | 40.1104457 
 longitude | -88.2330726 
 hours | [11:30-14:30, 11:30-14:30,,, 11:30-14:30, 11:30-14:30, 11:30-14:30] 
 categories | [Ethnic Food, Food Trucks, Specialty Food, Imported Food, Argentine, Food, Restaurants, Empanadas] 
-RECORD 1-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | CsLQLiRoafpJPJSkNX2h5Q 
 name | Middle East Deli 
 state | NC 
 city | Charlotte 
 attributes | [,,,,,,,,, True, {'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False},,,,,,,,, True,,,,,,,, False, u'casual',,, True, 1, False,, True,,,] 
 postal_code | 28205 
 latitude | 35.194894 
 longitude | -80.767442 
 hours | null 
 categories | [Food, Restaurants, Grocery, Middle Eastern] 
-RECORD 2-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | eBEfgOPG7pvFhb2wcG9I7w 
 name | Philthy Phillys 
 state | ON 
 city | Aurora 
 attributes | [,, u'none',,,,, True,,, {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False},, True,,,, False,,,,,,, True,,,, False,,, True,,, False, False, True,,, u'no'] 
 postal_code | L4G 7J1 
 latitude | 44.0109618 
 longitude | -79.448677 
 hours | [11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-21:0, 11:0-22:0, 11:0-22:0, 11:0-22:0] 
 categories | [Restaurants, Cheesesteaks, Poutineries] 
-RECORD 3--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
df_RestnBars.write.mode("overwrite").saveAsTable("RestnBars")
df_RestnBars.createOrReplaceTempView("RestnBars")

### Identifying business data with review data
Using `business_id`, we join together `df_RestnBars` and `busReviews` to get a table containing all the relevant information to the question. This table has more entries as each record lists the restaurants information for each different year that we have the data for. 

Using this version, we save the table as`RestnBars_with_Reviews` so that we can access it in Tableau and aid our visualization.

In [0]:
df_restnbars_reviews_count = spark.sql("""
SELECT U.business_id, U.name, U.state, U.city,
       U.attributes, U.postal_code, U.latitude, U.longitude, R.business_id as rBusiness_id, R.review_year, R.review_count
FROM RestnBars AS U left JOIN busReviews AS R
ON U.business_id = R.business_id
""")

print("Total number of businesses entries:" , df_restnbars_reviews_count.count())
df_restnbars_reviews_count.show(truncate=False, vertical = True)

Total number of businesses entries: 382753
-RECORD 0--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | --9e1ONYQuAa-CB_Rrw7Tw 
 name | Delmonico Steakhouse 
 state | NV 
 city | Las Vegas 
 attributes | [,, 'full_bar', {'romantic': False, 'intimate': False, 'classy': True, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': True, 'casual': False},, 'yes_corkage',, False,, True, {'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': True},, False,, True,, False,,, False, {'dessert': False, 'latenight': False, 'lunch': False, 'dinner': True, 'brunch': False, 'breakfast': False},, False, True,, u'average',, False, 'dressy',, False, True, 4, True,, False,,, u'no'] 
 postal_code | 89109 
 latitude | 36.123183 
 longitude | -115.16919 
 rBusiness_id | --9e1ONYQuAa-CB_Rrw7Tw 
 review_year | 2010 
 review_count | 133 
-RECORD 1--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | --9e1ONYQuAa-CB_Rrw7Tw 
 name | Delmonico Steakhouse 
 state | NV 
 city | Las Vegas 
 attributes | [,, 'full_bar', {'romantic': False, 'intimate': False, 'classy': True, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': True, 'casual': False},, 'yes_corkage',, False,, True, {'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': True},, False,, True,, False,,, False, {'dessert': False, 'latenight': False, 'lunch': False, 'dinner': True, 'brunch': False, 'breakfast': False},, False, True,, u'average',, False, 'dressy',, False, True, 4, True,, False,,, u'no'] 
 postal_code | 89109 
 latitude | 36.123183 
 longitude | -115.16919 
 rBusiness_id | --9e1ONYQuAa-CB_Rrw7Tw 
 review_year | 2011 
 review_count | 165 
-RECORD 2--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 business_id | --9e1ONYQuAa-CB_Rrw7Tw 
 name | Delmonico Steakhouse 
 state | NV 
 city | Las Vegas 
 attributes | [,, 'full_bar', {'romantic': False, 'intimate': False, 'classy': True, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': True, 'casual': False},, 'yes_corkage',, False,, True, {'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': True},, False,, True,, False,,, False, {'dessert': False, 'latenight': False, 'lunch': False, 'dinner': True, 'brunch': False, 'breakfast': False},, False, True,, u'average',, False, 'dressy',, False, True, 4, True,, False,,, u'no'] 
 postal_code | 89109 
 latitude | 36.123183 
 longitude | -115.16919 
 rBusiness_id | --9e1ONYQuAa-CB_Rrw7Tw 
 review_year | 2013 
 review_count | 165 


In [0]:
df_restnbars_reviews_count.write.mode("overwrite").saveAsTable("RestnBars_with_Reviews")

df_restnbars_reviews_count.createOrReplaceTempView("RestnBarsReview")

# Working with Hours
We want to know if the number of hours reported per week by a restaurant has any relationship with the number of reviews they receive from users. With this data wrangling and transformation we can see if other time-factors such as the year and the day of the week will also influence a restaurant's review count.

### Identifying the restaurants that didn't list their hours 

To initiate the analysis, we need to identify how many of the restaurants that don't provide their hours from `df_RestnBars`.

In [0]:
spark.sql("""
SELECT hours, COUNT(business_id) AS count
FROM RestnBars
WHERE hours IS NULL
GROUP BY hours
""").show()


+-----+-----+
hours|count|
+-----+-----+
 null|14583|
+-----+-----+

#### Removing the restaurants that do not provide hours
The restaurants that didn't have their hours listed are removed from `df_RestnBars` as we want to work with restaurants that provides us the hours for effective analysis.

In [0]:
df_yesHours = spark.sql("""
SELECT business_id, name, state, hours
FROM RestnBars
WHERE hours IS NOT NULL
""")
print("number of businesses with hours:", df_yesHours.count())
df_yesHours.show(truncate = False)
df_yesHours.createOrReplaceTempView("have_hours")

number of businesses with hours: 54402
+----------------------+-------------------------------+-----+-----------------------------------------------------------------------------------+
business_id |name |state|hours |
+----------------------+-------------------------------+-----+-----------------------------------------------------------------------------------+
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |IL |[11:30-14:30, 11:30-14:30,,, 11:30-14:30, 11:30-14:30, 11:30-14:30] |
eBEfgOPG7pvFhb2wcG9I7w|Philthy Phillys |ON |[11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-21:0, 11:0-22:0, 11:0-22:0, 11:0-22:0] |
9sRGfSVEfLhN_km60YruTA|Apadana Restaurant |ON |[12:0-0:0,, 12:0-0:0, 11:0-21:0, 12:0-21:0, 12:0-21:0, 12:0-21:0] |
vjTVxnsQEZ34XjYNS-XUpA|Wetzel's Pretzels |AZ |[10:0-21:0, 10:0-21:0, 10:0-21:0, 11:0-18:0, 10:0-21:0, 10:0-21:0, 10:0-21:0] |
98hyK2QEUeI8v2y0AghfZA|Pho Lee's Vietnamese Restaurant|OH |[11:0-20:0, 11:0-20:0, 11:0-20:0, 11:0-20:0, 11:0-20:0,, 11:0-20:0] |
fhNf_sg-XzZ3e7HEVGuOZg|Meat Chix And Wieners |NV |[11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0] |
Ga2Bt7xfqoggTypWD5VpoQ|Amando's Bros |AZ |[7:0-20:0, 7:0-20:0, 7:0-20:0, 7:0-20:0, 7:0-20:0, 7:0-20:0, 7:0-20:0] |
tLpkSwdtqqoXwU0JAGnApw|Wendy's |OH |[10:0-3:0, 10:0-3:0, 10:0-3:0, 10:0-3:0, 10:0-3:0, 10:0-3:0, 10:0-3:0] |
Mmd5WDFq9hHcQ3uClngGjQ|Irene's Tap Room |AZ |[12:0-2:0, 0:0-0:0, 12:0-2:0, 12:0-0:0, 15:0-0:0, 15:0-0:0, 15:0-0:0] |
lK-wuiq8b1TuU7bfbQZgsg|Hingetown |OH |[0:0-0:0, 0:0-0:0, 0:0-0:0, 0:0-0:0, 0:0-0:0, 0:0-0:0, 0:0-0:0] |
-qjn24n8HYF6It9GQrQntw|Five Guys Burgers and Fries |NC |[11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0] |
ZkzutF0P_u0C0yTulwaHkA|Lelulos Pizzeria |PA |[11:0-22:0, 0:0-0:0, 11:0-22:0, 11:0-20:0, 11:0-21:0, 11:0-21:0, 11:0-21:0] |
0QjROMVW9ACKjhSEfHqNCQ|Mi Mi Restaurant |ON |[11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 12:0-22:0] |
OLciBerMDmD_WYzfYbmkqA|The Lounge at Saxe Theater |NV |[14:0-23:30, 14:0-23:30, 14:0-23:30, 14:0-21:0, 14:0-23:30, 14:0-23:30, 14:0-23:30]|
RrapAhd8ZxCj-iue7fu9FA|Ganga Restaurant |AB |[17:0-22:0, 17:0-22:0, 17:0-22:0,, 17:0-22:0, 17:0-22:0, 17:0-22:0] |
7j0kor_fkeYhyEpXh4OpnQ|Scramblers - Strongsville |OH |[6:30-15:0, 6:30-15:0, 6:30-15:0, 6:30-15:0, 6:30-15:0, 6:30-15:0, 6:30-15:0] |
OWkS1FXNJbozn-qPg3LWxg|Mama Napoli Pizza |NV |[18:0-0:0,,,,,,] |
j9bWpCRwpDVfwVT_V85qeA|Papaya Thai |AZ |[11:0-21:30, 11:0-21:0, 11:0-21:30, 12:0-21:0, 11:0-21:0, 11:0-21:0, 11:0-21:0] |
6GHwgKNlvfIMUpFaxgBjUA|L'Artisan Piadineria |QC |[9:0-17:0, 9:0-17:0,,, 9:0-17:0, 9:0-17:0, 9:0-17:0] |
Ir_QIzs-4o9ElOtiGuxJrw|Torteria Lupita |QC |[12:0-20:0,, 10:0-17:0,, 12:0-20:0, 12:0-20:0, 12:0-20:0] |
+----------------------+-------------------------------+-----+-----------------------------------------------------------------------------------+
only showing top 20 rows

### Listing hours of operation by each day of the week
The restaurant information(ID,name) and their hours of operation are queried to get data about the how the restaurant hours might vary in a particular day of the week.

In [0]:
df_colHours = spark.sql("""
SELECT business_id, name, hours.monday, hours.tuesday, hours.wednesday, hours.thursday, hours.friday, hours.saturday, hours.sunday
FROM have_hours
""")
print("Total number of businesses:", df_colHours.count())
df_colHours.show()
df_colHours.createOrReplaceTempView("colHours")

Total number of businesses: 54402
+--------------------+--------------------+-----------+-----------+-----------+-----------+-----------+----------+---------+
 business_id| name| monday| tuesday| wednesday| thursday| friday| saturday| sunday|
+--------------------+--------------------+-----------+-----------+-----------+-----------+-----------+----------+---------+
pQeaRpvuhoEqudo3u...| The Empanadas House|11:30-14:30|11:30-14:30|11:30-14:30|11:30-14:30|11:30-14:30| null| null|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0|11:0-21:0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| null| 12:0-21:0| 12:0-21:0| 12:0-21:0| 12:0-0:0| 12:0-0:0|11:0-21:0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| 10:0-21:0| 10:0-21:0| 10:0-21:0| 10:0-21:0| 10:0-21:0| 10:0-21:0|11:0-18:0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| 11:0-20:0| null| 11:0-20:0| 11:0-20:0| 11:0-20:0| 11:0-20:0|11:0-20:0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0|11:0-22:0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| 7:0-20:0| 7:0-20:0| 7:0-20:0| 7:0-20:0| 7:0-20:0| 7:0-20:0| 7:0-20:0|
tLpkSwdtqqoXwU0JA...| Wendy's| 10:0-3:0| 10:0-3:0| 10:0-3:0| 10:0-3:0| 10:0-3:0| 10:0-3:0| 10:0-3:0|
Mmd5WDFq9hHcQ3uCl...| Irene's Tap Room| 0:0-0:0| 15:0-0:0| 15:0-0:0| 15:0-0:0| 12:0-2:0| 12:0-2:0| 12:0-0:0|
lK-wuiq8b1TuU7bfb...| Hingetown| 0:0-0:0| 0:0-0:0| 0:0-0:0| 0:0-0:0| 0:0-0:0| 0:0-0:0| 0:0-0:0|
-qjn24n8HYF6It9GQ...|Five Guys Burgers...| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0|11:0-22:0|
ZkzutF0P_u0C0yTul...| Lelulos Pizzeria| 0:0-0:0| 11:0-21:0| 11:0-21:0| 11:0-21:0| 11:0-22:0| 11:0-22:0|11:0-20:0|
0QjROMVW9ACKjhSEf...| Mi Mi Restaurant| 11:0-22:0| 11:0-22:0| 12:0-22:0| 11:0-22:0| 11:0-22:0| 11:0-22:0|11:0-22:0|
OLciBerMDmD_WYzfY...|The Lounge at Sax...| 14:0-23:30| 14:0-23:30| 14:0-23:30| 14:0-23:30| 14:0-23:30|14:0-23:30|14:0-21:0|
RrapAhd8ZxCj-iue7...| Ganga Restaurant| 17:0-22:0| 17:0-22:0| 17:0-22:0| 17:0-22:0| 17:0-22:0| 17:0-22:0| null|
7j0kor_fkeYhyEpXh...|Scramblers - Stro...| 6:30-15:0| 6:30-15:0| 6:30-15:0| 6:30-15:0| 6:30-15:0| 6:30-15:0|6:30-15:0|
OWkS1FXNJbozn-qPg...| Mama Napoli Pizza| null| null| null| null| 18:0-0:0| null| null|
j9bWpCRwpDVfwVT_V...| Papaya Thai| 11:0-21:0| 11:0-21:0| 11:0-21:0| 11:0-21:0| 11:0-21:30|11:0-21:30|12:0-21:0|
6GHwgKNlvfIMUpFax...|L'Artisan Piadineria| 9:0-17:0| 9:0-17:0| 9:0-17:0| 9:0-17:0| 9:0-17:0| null| null|
Ir_QIzs-4o9ElOtiG...| Torteria Lupita| null| 12:0-20:0| 12:0-20:0| 12:0-20:0| 12:0-20:0| 10:0-17:0| null|
+--------------------+--------------------+-----------+-----------+-----------+-----------+-----------+----------+---------+
only showing top 20 rows

#### Organizing Hours

After identifying the hours per weekday for a restaurant, we group the week in an array to see how how many days the restaurant is open during a week. After, we establish a weekmap that has the corresponding hours a restaurant or bar is open for each day of the week.

In [0]:
spark.sql("""
SELECT business_id, name, ARRAY(Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday) AS hours
FROM colHours
""").show(5,truncate=False)

+----------------------+-------------------------------+-----------------------------------------------------------------------------+
business_id |name |hours |
+----------------------+-------------------------------+-----------------------------------------------------------------------------+
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |[11:30-14:30, 11:30-14:30, 11:30-14:30, 11:30-14:30, 11:30-14:30,,] |
eBEfgOPG7pvFhb2wcG9I7w|Philthy Phillys |[11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-22:0, 11:0-21:0]|
9sRGfSVEfLhN_km60YruTA|Apadana Restaurant |[, 12:0-21:0, 12:0-21:0, 12:0-21:0, 12:0-0:0, 12:0-0:0, 11:0-21:0] |
vjTVxnsQEZ34XjYNS-XUpA|Wetzel's Pretzels |[10:0-21:0, 10:0-21:0, 10:0-21:0, 10:0-21:0, 10:0-21:0, 10:0-21:0, 11:0-18:0]|
98hyK2QEUeI8v2y0AghfZA|Pho Lee's Vietnamese Restaurant|[11:0-20:0,, 11:0-20:0, 11:0-20:0, 11:0-20:0, 11:0-20:0, 11:0-20:0] |
+----------------------+-------------------------------+-----------------------------------------------------------------------------+
only showing top 5 rows

In [0]:
df_hours_maps = spark.sql("""
SELECT business_id, name, MAP("Monday",hours.Monday) AS map_monday,
                          MAP("Tuesday",hours.Tuesday) AS map_tuesday,
                          MAP("Wednesday",hours.Wednesday) AS map_wednesday,
                          MAP("Thursday",hours.Thursday) AS map_thursday,
                          MAP("Friday",hours.Friday) AS map_friday,
                          MAP("Saturday",hours.Saturday) AS map_saturday,
                          MAP("Sunday",hours.Sunday) AS map_sunday
FROM RestnBars
WHERE hours IS NOT NULL""")
df_hours_maps.show(5,truncate=False)
df_hours_maps.createOrReplaceTempView("hours_map")

+----------------------+-------------------------------+-----------------------+------------------------+--------------------------+-------------------------+-----------------------+-----------------------+---------------------+
business_id |name |map_monday |map_tuesday |map_wednesday |map_thursday |map_friday |map_saturday |map_sunday |
+----------------------+-------------------------------+-----------------------+------------------------+--------------------------+-------------------------+-----------------------+-----------------------+---------------------+
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |[Monday -> 11:30-14:30]|[Tuesday -> 11:30-14:30]|[Wednesday -> 11:30-14:30]|[Thursday -> 11:30-14:30]|[Friday -> 11:30-14:30]|[Saturday ->] |[Sunday ->] |
eBEfgOPG7pvFhb2wcG9I7w|Philthy Phillys |[Monday -> 11:0-22:0] |[Tuesday -> 11:0-22:0] |[Wednesday -> 11:0-22:0] |[Thursday -> 11:0-22:0] |[Friday -> 11:0-22:0] |[Saturday -> 11:0-22:0]|[Sunday -> 11:0-21:0]|
9sRGfSVEfLhN_km60YruTA|Apadana Restaurant |[Monday ->] |[Tuesday -> 12:0-21:0] |[Wednesday -> 12:0-21:0] |[Thursday -> 12:0-21:0] |[Friday -> 12:0-0:0] |[Saturday -> 12:0-0:0] |[Sunday -> 11:0-21:0]|
vjTVxnsQEZ34XjYNS-XUpA|Wetzel's Pretzels |[Monday -> 10:0-21:0] |[Tuesday -> 10:0-21:0] |[Wednesday -> 10:0-21:0] |[Thursday -> 10:0-21:0] |[Friday -> 10:0-21:0] |[Saturday -> 10:0-21:0]|[Sunday -> 11:0-18:0]|
98hyK2QEUeI8v2y0AghfZA|Pho Lee's Vietnamese Restaurant|[Monday -> 11:0-20:0] |[Tuesday ->] |[Wednesday -> 11:0-20:0] |[Thursday -> 11:0-20:0] |[Friday -> 11:0-20:0] |[Saturday -> 11:0-20:0]|[Sunday -> 11:0-20:0]|
+----------------------+-------------------------------+-----------------------+------------------------+--------------------------+-------------------------+-----------------------+-----------------------+---------------------+
only showing top 5 rows

In [0]:
df_week_map = spark.sql("""
SELECT business_id, name, MAP_CONCAT(map_monday,map_tuesday,map_wednesday,map_thursday,map_friday,map_saturday,map_sunday) AS week_map
FROM hours_map
""")
df_week_map.show(5,truncate=False)
df_week_map.createOrReplaceTempView("week_map")

+----------------------+-------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
business_id |name |week_map |
+----------------------+-------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |[Monday -> 11:30-14:30, Tuesday -> 11:30-14:30, Wednesday -> 11:30-14:30, Thursday -> 11:30-14:30, Friday -> 11:30-14:30, Saturday ->, Sunday ->] |
eBEfgOPG7pvFhb2wcG9I7w|Philthy Phillys |[Monday -> 11:0-22:0, Tuesday -> 11:0-22:0, Wednesday -> 11:0-22:0, Thursday -> 11:0-22:0, Friday -> 11:0-22:0, Saturday -> 11:0-22:0, Sunday -> 11:0-21:0]|
9sRGfSVEfLhN_km60YruTA|Apadana Restaurant |[Monday ->, Tuesday -> 12:0-21:0, Wednesday -> 12:0-21:0, Thursday -> 12:0-21:0, Friday -> 12:0-0:0, Saturday -> 12:0-0:0, Sunday -> 11:0-21:0] |
vjTVxnsQEZ34XjYNS-XUpA|Wetzel's Pretzels |[Monday -> 10:0-21:0, Tuesday -> 10:0-21:0, Wednesday -> 10:0-21:0, Thursday -> 10:0-21:0, Friday -> 10:0-21:0, Saturday -> 10:0-21:0, Sunday -> 11:0-18:0]|
98hyK2QEUeI8v2y0AghfZA|Pho Lee's Vietnamese Restaurant|[Monday -> 11:0-20:0, Tuesday ->, Wednesday -> 11:0-20:0, Thursday -> 11:0-20:0, Friday -> 11:0-20:0, Saturday -> 11:0-20:0, Sunday -> 11:0-20:0] |
+----------------------+-------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 5 rows

#### Finding the opening and closing times for the restaurants
To breakdown the hours of operation to get the total hours open for a restaurant, we seperate the hours column into two columns: one for when the restaurant opens and the other for when it closes.

In [0]:
df_weekly = spark.sql("""
SELECT business_id, name, EXPLODE(week_map) AS (weekday,hours), SPLIT(hours,"\\\-")[0] AS open, SPLIT(hours,"\\\-")[1] AS close
FROM week_map
""")
df_weekly.show(20)
df_weekly.createOrReplaceTempView("weekly")

+--------------------+-------------------+---------+-----------+-----+-----+
 business_id| name| weekday| hours| open|close|
+--------------------+-------------------+---------+-----------+-----+-----+
pQeaRpvuhoEqudo3u...|The Empanadas House| Monday|11:30-14:30|11:30|14:30|
pQeaRpvuhoEqudo3u...|The Empanadas House| Tuesday|11:30-14:30|11:30|14:30|
pQeaRpvuhoEqudo3u...|The Empanadas House|Wednesday|11:30-14:30|11:30|14:30|
pQeaRpvuhoEqudo3u...|The Empanadas House| Thursday|11:30-14:30|11:30|14:30|
pQeaRpvuhoEqudo3u...|The Empanadas House| Friday|11:30-14:30|11:30|14:30|
pQeaRpvuhoEqudo3u...|The Empanadas House| Saturday| null| null| null|
pQeaRpvuhoEqudo3u...|The Empanadas House| Sunday| null| null| null|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Monday| 11:0-22:0| 11:0| 22:0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Tuesday| 11:0-22:0| 11:0| 22:0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys|Wednesday| 11:0-22:0| 11:0| 22:0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Thursday| 11:0-22:0| 11:0| 22:0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Friday| 11:0-22:0| 11:0| 22:0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Saturday| 11:0-22:0| 11:0| 22:0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Sunday| 11:0-21:0| 11:0| 21:0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Monday| null| null| null|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Tuesday| 12:0-21:0| 12:0| 21:0|
9sRGfSVEfLhN_km60...| Apadana Restaurant|Wednesday| 12:0-21:0| 12:0| 21:0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Thursday| 12:0-21:0| 12:0| 21:0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Friday| 12:0-0:0| 12:0| 0:0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Saturday| 12:0-0:0| 12:0| 0:0|
+--------------------+-------------------+---------+-----------+-----+-----+
only showing top 20 rows

#### Further splitting to get the minutes for opening and closing
Since not all businesses open or close on the hour, meaning that they may open/close at 10:30 or 10:45, etc., we need to further split the opening and closing times and create a column that specificly identifies the minutes. By doing this, we can better calculate the total number of hours a business is operating.

In [0]:
df_weekly1 = spark.sql("""
SELECT business_id, name, weekday, hours, 
SPLIT(open,"\\\:")[0] AS open_hours, SPLIT(open,"\\\:")[1] AS open_minutes,
SPLIT(close,"\\\:")[0] AS close_hours, SPLIT(close,"\\\:")[1] AS close_minutes
FROM weekly
WHERE hours IS NOT NULL""")
df_weekly1.show(60)
df_weekly1.createOrReplaceTempView("weekly1")

+--------------------+--------------------+---------+-----------+----------+------------+-----------+-------------+
 business_id| name| weekday| hours|open_hours|open_minutes|close_hours|close_minutes|
+--------------------+--------------------+---------+-----------+----------+------------+-----------+-------------+
pQeaRpvuhoEqudo3u...| The Empanadas House| Monday|11:30-14:30| 11| 30| 14| 30|
pQeaRpvuhoEqudo3u...| The Empanadas House| Tuesday|11:30-14:30| 11| 30| 14| 30|
pQeaRpvuhoEqudo3u...| The Empanadas House|Wednesday|11:30-14:30| 11| 30| 14| 30|
pQeaRpvuhoEqudo3u...| The Empanadas House| Thursday|11:30-14:30| 11| 30| 14| 30|
pQeaRpvuhoEqudo3u...| The Empanadas House| Friday|11:30-14:30| 11| 30| 14| 30|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Monday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Tuesday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys|Wednesday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Thursday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Friday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Saturday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Sunday| 11:0-21:0| 11| 0| 21| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Tuesday| 12:0-21:0| 12| 0| 21| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant|Wednesday| 12:0-21:0| 12| 0| 21| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Thursday| 12:0-21:0| 12| 0| 21| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Friday| 12:0-0:0| 12| 0| 0| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Saturday| 12:0-0:0| 12| 0| 0| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Sunday| 11:0-21:0| 11| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Monday| 10:0-21:0| 10| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Tuesday| 10:0-21:0| 10| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels|Wednesday| 10:0-21:0| 10| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Thursday| 10:0-21:0| 10| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Friday| 10:0-21:0| 10| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Saturday| 10:0-21:0| 10| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Sunday| 11:0-18:0| 11| 0| 18| 0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Monday| 11:0-20:0| 11| 0| 20| 0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...|Wednesday| 11:0-20:0| 11| 0| 20| 0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Thursday| 11:0-20:0| 11| 0| 20| 0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Friday| 11:0-20:0| 11| 0| 20| 0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Saturday| 11:0-20:0| 11| 0| 20| 0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Sunday| 11:0-20:0| 11| 0| 20| 0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Monday| 11:0-22:0| 11| 0| 22| 0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Tuesday| 11:0-22:0| 11| 0| 22| 0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...|Wednesday| 11:0-22:0| 11| 0| 22| 0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Thursday| 11:0-22:0| 11| 0| 22| 0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Friday| 11:0-22:0| 11| 0| 22| 0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Saturday| 11:0-22:0| 11| 0| 22| 0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Sunday| 11:0-22:0| 11| 0| 22| 0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Monday| 7:0-20:0| 7| 0| 20| 0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Tuesday| 7:0-20:0| 7| 0| 20| 0|
Ga2Bt7xfqoggTypWD...| Amando's Bros|Wednesday| 7:0-20:0| 7| 0| 20| 0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Thursday| 7:0-20:0| 7| 0| 20| 0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Friday| 7:0-20:0| 7| 0| 20| 0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Saturday| 7:0-20:0| 7| 0| 20| 0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Sunday| 7:0-20:0| 7| 0| 20| 0|
tLpkSwdtqqoXwU0JA...| Wendy's| Monday| 10:0-3:0| 10| 0| 3| 0|
tLpkSwdtqqoXwU0JA...| Wendy's| Tuesday| 10:0-3:0| 10| 0| 3| 0|
tLpkSwdtqqoXwU0JA...| Wendy's|Wednesday| 10:0-3:0| 10| 0| 3| 0|
tLpkSwdtqqoXwU0JA...| Wendy's| Thursday| 10:0-3:0| 10| 0| 3| 0|
tLpkSwdtqqoXwU0JA...| Wendy's| Friday| 10:0-3:0| 10| 0| 

#### Converting time data fields into numerical data types
Since the time data is stored as a string, we need to cast the times as an integer so that the total number of hours open can be calculated.

In [0]:
df_weekly2 = spark.sql("""
select business_id, name, weekday, hours, cast(open_hours as int), cast(open_minutes as int), cast(close_hours as int), cast(close_minutes as int) 
from weekly1
""")
df_weekly2.show()
df_weekly2.createOrReplaceTempView("weeklyTimeSplit")

+--------------------+-------------------+---------+-----------+----------+------------+-----------+-------------+
 business_id| name| weekday| hours|open_hours|open_minutes|close_hours|close_minutes|
+--------------------+-------------------+---------+-----------+----------+------------+-----------+-------------+
pQeaRpvuhoEqudo3u...|The Empanadas House| Monday|11:30-14:30| 11| 30| 14| 30|
pQeaRpvuhoEqudo3u...|The Empanadas House| Tuesday|11:30-14:30| 11| 30| 14| 30|
pQeaRpvuhoEqudo3u...|The Empanadas House|Wednesday|11:30-14:30| 11| 30| 14| 30|
pQeaRpvuhoEqudo3u...|The Empanadas House| Thursday|11:30-14:30| 11| 30| 14| 30|
pQeaRpvuhoEqudo3u...|The Empanadas House| Friday|11:30-14:30| 11| 30| 14| 30|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Monday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Tuesday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys|Wednesday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Thursday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Friday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Saturday| 11:0-22:0| 11| 0| 22| 0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Sunday| 11:0-21:0| 11| 0| 21| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Tuesday| 12:0-21:0| 12| 0| 21| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant|Wednesday| 12:0-21:0| 12| 0| 21| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Thursday| 12:0-21:0| 12| 0| 21| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Friday| 12:0-0:0| 12| 0| 0| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Saturday| 12:0-0:0| 12| 0| 0| 0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Sunday| 11:0-21:0| 11| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Monday| 10:0-21:0| 10| 0| 21| 0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Tuesday| 10:0-21:0| 10| 0| 21| 0|
+--------------------+-------------------+---------+-----------+----------+------------+-----------+-------------+
only showing top 20 rows

### Calculating Total Number of Hours Open
In this step, we calculate the total number of hours open for each business. First and foremost, we create a formula that determines how the final values will be presented, preventing any numbers from being negative. 

We then move to minutes, where we take `close_minutes` and subtract it by `open_minutes` to find how many minutes a business operates, excluding the hours column. To finish the calculation, we add together `HoursOpen` and `Minutes` to find the total number of hours a business operates. 

The result is stored in `calculated_table`.

In [0]:
df_calculate = spark.sql("""
SELECT business_id, name, weekday, hours, open_minutes, close_minutes,
CASE 
WHEN (close_hours - open_hours) > 0 THEN (close_hours - open_hours)
  WHEN (close_hours - open_hours) < 0 THEN ((close_hours-open_hours)*(-1))
  ELSE 24
  END AS HoursOpen
FROM weeklyTimeSplit
""")
df_calculate.createOrReplaceTempView("calculated1")

#Calculating Minutes open during the day
minutes_calc = spark.sql("""
SELECT business_id, name, weekday, hours, HoursOpen, ((close_minutes - open_minutes)/60) as Minutes
FROM calculated1
""")
minutes_calc.createOrReplaceTempView("minutes")

#Adding minutes open during the day to OpenHours
df_busHours = spark.sql("""
SELECT business_id, name, weekday, hours, (HoursOpen + Minutes) as OpenHours
FROM minutes
""")
df_busHours.createOrReplaceTempView("busHoursOpen")
df_busHours.show(50)
df_busHours.printSchema()

+--------------------+--------------------+---------+-----------+---------+
 business_id| name| weekday| hours|OpenHours|
+--------------------+--------------------+---------+-----------+---------+
pQeaRpvuhoEqudo3u...| The Empanadas House| Monday|11:30-14:30| 3.0|
pQeaRpvuhoEqudo3u...| The Empanadas House| Tuesday|11:30-14:30| 3.0|
pQeaRpvuhoEqudo3u...| The Empanadas House|Wednesday|11:30-14:30| 3.0|
pQeaRpvuhoEqudo3u...| The Empanadas House| Thursday|11:30-14:30| 3.0|
pQeaRpvuhoEqudo3u...| The Empanadas House| Friday|11:30-14:30| 3.0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Monday| 11:0-22:0| 11.0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Tuesday| 11:0-22:0| 11.0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys|Wednesday| 11:0-22:0| 11.0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Thursday| 11:0-22:0| 11.0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Friday| 11:0-22:0| 11.0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Saturday| 11:0-22:0| 11.0|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| Sunday| 11:0-21:0| 10.0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Tuesday| 12:0-21:0| 9.0|
9sRGfSVEfLhN_km60...| Apadana Restaurant|Wednesday| 12:0-21:0| 9.0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Thursday| 12:0-21:0| 9.0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Friday| 12:0-0:0| 12.0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Saturday| 12:0-0:0| 12.0|
9sRGfSVEfLhN_km60...| Apadana Restaurant| Sunday| 11:0-21:0| 10.0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Monday| 10:0-21:0| 11.0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Tuesday| 10:0-21:0| 11.0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels|Wednesday| 10:0-21:0| 11.0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Thursday| 10:0-21:0| 11.0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Friday| 10:0-21:0| 11.0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Saturday| 10:0-21:0| 11.0|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| Sunday| 11:0-18:0| 7.0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Monday| 11:0-20:0| 9.0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...|Wednesday| 11:0-20:0| 9.0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Thursday| 11:0-20:0| 9.0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Friday| 11:0-20:0| 9.0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Saturday| 11:0-20:0| 9.0|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| Sunday| 11:0-20:0| 9.0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Monday| 11:0-22:0| 11.0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Tuesday| 11:0-22:0| 11.0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...|Wednesday| 11:0-22:0| 11.0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Thursday| 11:0-22:0| 11.0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Friday| 11:0-22:0| 11.0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Saturday| 11:0-22:0| 11.0|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| Sunday| 11:0-22:0| 11.0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Monday| 7:0-20:0| 13.0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Tuesday| 7:0-20:0| 13.0|
Ga2Bt7xfqoggTypWD...| Amando's Bros|Wednesday| 7:0-20:0| 13.0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Thursday| 7:0-20:0| 13.0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Friday| 7:0-20:0| 13.0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Saturday| 7:0-20:0| 13.0|
Ga2Bt7xfqoggTypWD...| Amando's Bros| Sunday| 7:0-20:0| 13.0|
tLpkSwdtqqoXwU0JA...| Wendy's| Monday| 10:0-3:0| 7.0|
tLpkSwdtqqoXwU0JA...| Wendy's| Tuesday| 10:0-3:0| 7.0|
tLpkSwdtqqoXwU0JA...| Wendy's|Wednesday| 10:0-3:0| 7.0|
tLpkSwdtqqoXwU0JA...| Wendy's| Thursday| 10:0-3:0| 7.0|
tLpkSwdtqqoXwU0JA...| Wendy's| Friday| 10:0-3:0| 7.0|
+--------------------+--------------------+---------+-----------+---------+
only showing top 50 rows

root
-- business_id: string (nullable = true)
-- name: string (nullable = true)
-- weekday: string (nullable = false)
-- hours: string (nullable = true)
-- OpenHours: double (nullable = true)

In [0]:
df_calculate.createOrReplaceTempView("calculated_table")

#### Comparing the number of restaurants with total hours of operation
After finding the total hours open, we compare it with the total number of businesses that operate for a given length of time.

In [0]:
spark.sql("""
SELECT count(business_id) as TotalBus, OpenHours, weekday
FROM busHoursOpen
GROUP BY OpenHours, weekday
""").show()

+--------+------------------+---------+
TotalBus| OpenHours| weekday|
+--------+------------------+---------+
 184| 16.5|Wednesday|
 107| 17.5| Thursday|
 5412| 10.0| Saturday|
 755| 4.5|Wednesday|
 325| 18.0|Wednesday|
 14| 12.75| Friday|
 5980| 12.0| Friday|
 13|1.9833333333333334| Friday|
 1539| 9.5|Wednesday|
 2282| 11.5| Friday|
 33| 1.0| Monday|
 109| 17.5| Friday|
 25| 11.75| Saturday|
 1|10.166666666666666| Monday|
 6390| 10.0| Tuesday|
 240| 6.5| Tuesday|
 4| 7.25| Thursday|
 46| 2.5| Friday|
 268| 13.5| Saturday|
 717| 4.0| Saturday|
+--------+------------------+---------+
only showing top 20 rows

### Identifying business data with hours of operation and review data
To create a cohesive table with the review information, we join together `caclulated_table`, which has the total hours of operation, with `busReview`, which contains the review data including the years and review count. 
Using this, we can create the table `CalculatedReviews` and import it into Tableau to begin to visualize the data.

In [0]:
df_calculated_reviews = spark.sql("""
SELECT distinct ct.business_id, ct.name, sum(ct.hoursopen) as WeeklyHoursOpen , r.review_year as ReviewYear,r.review_count as ReviewCount 
FROM calculated_table ct inner join busReviews r on ct.business_id = r.business_id 
GROUP BY ct.business_id, ct.name,r.review_year, r.review_count 
ORDER BY ct.name
""")
df_calculated_reviews.show()
df_calculated_reviews.createOrReplaceTempView("calculated_reviews")

+--------------------+--------------------+---------------+----------+-----------+
 business_id| name|WeeklyHoursOpen|ReviewYear|ReviewCount|
+--------------------+--------------------+---------------+----------+-----------+
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2011| 6|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2013| 5|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2014| 5|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2016| 7|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2017| 6|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2015| 6|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2018| 2|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2019| 4|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2010| 9|
Tk79j0dMuCDF24NhX...| Fresh Madison Ma...| 42| 2012| 6|
42yLya7lhe7TQGz3K...|#1 Hawaiian Barbecue| 79| 2019| 9|
42yLya7lhe7TQGz3K...|#1 Hawaiian Barbecue| 79| 2015| 11|
42yLya7lhe7TQGz3K...|#1 Hawaiian Barbecue| 79| 2017| 8|
gA78_OGp1ekMvZWjR...|#1 Hawaiian Barbecue| 96| 2018| 23|
gA78_OGp1ekMvZWjR...|#1 Hawaiian Barbecue| 96| 2019| 13|
42yLya7lhe7TQGz3K...|#1 Hawaiian Barbecue| 79| 2016| 4|
42yLya7lhe7TQGz3K...|#1 Hawaiian Barbecue| 79| 2018| 8|
42yLya7lhe7TQGz3K...|#1 Hawaiian Barbecue| 79| 2014| 7|
AhD9x2NNFqPvVwmyE...| #1 Pho| 62| 2015| 1|
AhD9x2NNFqPvVwmyE...| #1 Pho| 62| 2010| 2|
+--------------------+--------------------+---------------+----------+-----------+
only showing top 20 rows

In [0]:
df_calculated_reviews.write.mode("overwrite").saveAsTable("CalculatedReviews")

## Do hours affect review count?
Based on the data that we wrangled, we developed 2 visualizations.
1. In the first visualization, the reviews count is being grouped by the total number of hours open. From the graph, we can see that on average, restaurants that are open for 11 hours tend to receive more reviews. Interestingly enough, the restaurants that are open for 24 hours on Monday, end up receiving the most reviews. 
2. In the second visualization, we take a more macro view of the hours and analyze it from a weekly standpoint. The data looks at yearly review counts, so as the time gets more recent, the number of reviews increase. Looking at the hours, regardless of the year, businesses that are open for 60-80 hours per week receive the highest number of reviews, then businesses that operate 80-100 hours receive the next highest number.

### <a href = "https://public.tableau.com/views/AmountofReviewsbyHoursOpened/AmountofReviewsreceivedperhour?:language=en&:display_count=y&publish=yes&:origin=viz_share_link">Visualization 1: Hours Opened by their Corresponding Weekday</a>

In [0]:
displayHTML("<div class='tableauPlaceholder' id='viz1607410806088' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Am&#47;AmountofReviewsbyHoursOpened&#47;AmountofReviewsreceivedperhour&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='AmountofReviewsbyHoursOpened&#47;AmountofReviewsreceivedperhour' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Am&#47;AmountofReviewsbyHoursOpened&#47;AmountofReviewsreceivedperhour&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1607410806088');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>")

### <a href= "https://public.tableau.com/views/ProfilingReviewsbyHoursOpened/ReviewsbasedontheWeeklyHoursOpened?:language=en&:display_count=y&:origin=viz_share_link">Visualization 2: Hours Opened by Their Corresponding Year</a>

In [0]:
displayHTML("<div class='tableauPlaceholder' id='viz1607320671164' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Pr&#47;ProfilingReviewsbyHoursOpened&#47;ReviewsbasedontheWeeklyHoursOpened&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='ProfilingReviewsbyHoursOpened&#47;ReviewsbasedontheWeeklyHoursOpened' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Pr&#47;ProfilingReviewsbyHoursOpened&#47;ReviewsbasedontheWeeklyHoursOpened&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /></object></div>                <script type='text/javascript'> var divElement = document.getElementById('viz1607320671164');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>")

#How do 'GoodforMeal' attributes compare in review counts?
* Restaurants on Yelp can list whether or not their business is good for certain meals like BreakFast, Brunch, Lunch, 
  Dinner or LateNight. When searching for restaurants to dine in using the Yelp app, you can check these filters to narrow your results. 
* We want to see whether restaurants that have these attributes listed have more reviews than restaurants that don't have it listed. 
* Collectively, we also want to see how many reviews restaurants GoodForMeal attribute with True, False, and None values receive.

To start, we will wrangle the data to split the business categories and get the `GoodForMeal` attribute
In the command below, we will extract only does records where their `attributes.GoodForMeal` is not NULL. 
>We'll be extracting the following attributes:
`business_id` `name` `GoodForMeal`
*  ***business_id*** is a unique id for every business on the Yelp dataset
*  ***GoodForMeal*** is an array for attribiutes we will be extracting, `breakfast` `brunch` `lunch` `dinner` `latenight` 
*  ***reviews*** is the total review count for that business, however, we will be using reviews from the `busReviews` view we created above.

In [0]:
df_good_for_meals1 = df_business.select("business_id", "name", f.split(df_business.attributes.goodformeal,'\s*,\s*').alias("goodformeal"), f.split(df_business.categories,'\s*,\s*').alias("categories"))

#Taking records that are categorized as Restaurants
df_good_for_meals = spark.sql("""
SELECT *
FROM goodforCopy
WHERE ARRAY_CONTAINS(categories, "Restaurants")
""")

In [0]:
df_mealsList = df_good_for_meals.select("business_id", "name","categories", df_good_for_meals.goodformeal[1].substr(13,6).alias("LateNight"),df_good_for_meals.goodformeal[2].substr(9,6).alias("Lunch"),df_good_for_meals.goodformeal[3].substr(10,6).alias("Dinner"),df_good_for_meals.goodformeal[4].alias("Brunch"),df_good_for_meals.goodformeal[5].alias("Breakfast"))

In [0]:
df_double = df_mealsList.select("business_id", "name", "LateNight", "Lunch", "Dinner", "Brunch", "Breakfast",df_mealsList.Brunch.alias("GoodForBrunch2"), df_mealsList.Breakfast.alias("GoodForBreakfast2"))

df_double.createOrReplaceTempView("double")
df_GoodForMealSplit.createOrReplaceTempView("GoodForMealSplit")

In [0]:
df_MealSplit = spark.sql("""
SELECT business_id, name, LateNight, Lunch, Dinner, Brunch, TRIM(TRAILING '}' from Breakfast) AS Breakfast, reviews
FROM GoodForMealSplit
""")
print("Total number of Businesses with a GoodForMeal Attribute:", df_MealSplit.count())

df_MealSplit.createOrReplaceTempView("MealSplit")

Total number of Businesses with a GoodForMeal Attribute: 222922

Here we are checking if we have extracted the right values from the `goodformeal` attributes
**df_TotalLateNight** dataframe will store our query that sums total number of restaurants with values None, False, null, True for their GoodForLateNight attribute
There are 196,152 restaurants that list their business as False for GoodForLateNight

In [0]:
df_TotalLateNight = spark.sql("""
SELECT LateNight, count(business_id) as BusCount
FROM double
GROUP BY LateNight
""").show()

+---------+--------+
LateNight|BusCount|
+---------+--------+
 None| 1421|
 True| 1868|
 null| 29536|
 False| 31119|
+---------+--------+

Checking for values in the goodforBrunch column, we see that we have values that belong in the goodforBreakfast column. We can tell because it says 'breakfast' following the value.
Printing values for Breakfast confirms that we have have some more cleaning to do.
> **91 values** from Brunch and Breakfast columns are switched

In [0]:
df_Brunch = spark.sql("""
SELECT Brunch, count(business_id) as count
FROM double
GROUP BY Brunch
""")
df_Brunch.show()
df_Breakfast = spark.sql("""
SELECT Breakfast, count(business_id) as count
FROM double
GROUP BY Breakfast
""")
df_Breakfast.show()

+------------------+-----+
 Brunch|count|
+------------------+-----+
 null|29536|
 'brunch': None| 1505|
'breakfast': False| 89|
 'brunch': True| 3509|
 'brunch': False|29304|
 'breakfast': True| 1|
+------------------+-----+

+-------------------+-----+
 Breakfast|count|
+-------------------+-----+
 null|29536|
 'breakfast': None}| 901|
 'brunch': False}| 87|
 'breakfast': True}| 3236|
'breakfast': False}|30181|
 'brunch': True}| 3|
+-------------------+-----+

In [0]:
#Taking the actual goodforBrunch value from column goodforBreakfast2
df_swap1 = df_double.withColumn("Brunch",
          f.when(df_double.Brunch.substr(2,5) == 'break', df_double.GoodForBreakfast2).otherwise(df_double.Brunch))

#Taking the actual goodforBreakfast value from column goodforBrunch2
df_swap2= df_swap1.withColumn("Breakfast",
          f.when(df_swap1.Breakfast.substr(2,6) == 'brunch', df_swap1.GoodForBrunch2).otherwise(df_swap1.Breakfast))

#Getting columns we need and Cleaning up so that the only values we have are either True, False, None, or NULL in the goodforBrunch column
df_brunch = df_swap2.select("business_id", "name", "LateNight", "Dinner", "Lunch", "Breakfast", df_swap2.Brunch.substr(10,6).alias("Brunch"))

#Removing bracket on column values 
df_brunch1 = df_brunch.withColumn("Brunch",
          f.when(df_brunch.Brunch.substr(-1,1) == '}', 'True').otherwise(df_brunch.Brunch))

#Removing whitespace on the leftside 
df_brunch2 = df_brunch1.withColumn("Brunch", f.ltrim(df_brunch1.Brunch))

In [0]:
#Cleaning up so that the only values we have are either True, False, None, or Null in the goodforBreakfast column.
df_breakfast1 = df_brunch2.select("business_id", "name", "LateNight","Lunch" ,"Dinner", "Brunch",df_brunch2.Breakfast.substr(13,6).alias("Breakfast"))

#Removing whitespace on leftside
df_breakfast2 = df_breakfast1.withColumn("Breakfast", f.ltrim(df_breakfast1.Breakfast))

#Removing brackets from values
df_breakfast3 = df_breakfast2.withColumn("Breakfast", f.regexp_replace(df_breakfast2.Breakfast, "}", "").alias("Breakfast"))

#Creating a new dataframe to use for tablaeu and create a view
df_framer = df_breakfast3
df_framer.createOrReplaceTempView("vis")

In [0]:
spark.sql("""
select Brunch, count(business_id)
from vis
group by Brunch""").show()

spark.sql("""
select Breakfast, count(business_id)
from vis
group by Breakfast""").show()

###Identifying businesses, review, and hour data
Here we will join tables `busReviews` and `busHoursOpen`
>* We will select ***`business_id`, `name`, `weekday`, `review_count`, `review_year`, `OpenHours`***
>**`review_year`** will be useful in categorizing review_counts when visualizing the data

In [0]:
df_insight1 = spark.sql("""
SELECT H.business_id, H.name, H.weekday,  R.review_count, R.review_year, H.OpenHours
FROM busReviews AS R INNER JOIN busHoursOpen AS H
ON R.business_id = H.business_id
""")
df_insight1.createOrReplaceTempView("insight1")
df_insight1.show()


+--------------------+--------------------+---------+------------+-----------+---------+
 business_id| name| weekday|review_count|review_year|OpenHours|
+--------------------+--------------------+---------+------------+-----------+---------+
P2dFr-Jg1OUoNMika...| Café Hollywood| Sunday| 118| 2019| 24.0|
P2dFr-Jg1OUoNMika...| Café Hollywood| Saturday| 118| 2019| 24.0|
P2dFr-Jg1OUoNMika...| Café Hollywood| Friday| 118| 2019| 24.0|
P2dFr-Jg1OUoNMika...| Café Hollywood| Thursday| 118| 2019| 24.0|
P2dFr-Jg1OUoNMika...| Café Hollywood|Wednesday| 118| 2019| 24.0|
P2dFr-Jg1OUoNMika...| Café Hollywood| Tuesday| 118| 2019| 24.0|
P2dFr-Jg1OUoNMika...| Café Hollywood| Monday| 118| 2019| 24.0|
BjH8Xepc10i6OhCDQ...|Emeril's New Orle...| Sunday| 136| 2018| 10.5|
BjH8Xepc10i6OhCDQ...|Emeril's New Orle...| Saturday| 136| 2018| 10.5|
BjH8Xepc10i6OhCDQ...|Emeril's New Orle...| Friday| 136| 2018| 10.5|
BjH8Xepc10i6OhCDQ...|Emeril's New Orle...| Thursday| 136| 2018| 10.5|
BjH8Xepc10i6OhCDQ...|Emeril's New Orle...|Wednesday| 136| 2018| 10.5|
BjH8Xepc10i6OhCDQ...|Emeril's New Orle...| Tuesday| 136| 2018| 10.5|
BjH8Xepc10i6OhCDQ...|Emeril's New Orle...| Monday| 136| 2018| 10.5|
5Q95IfYIbP6vXz4wj...|Applebee's Grill ...| Sunday| 47| 2019| 11.0|
5Q95IfYIbP6vXz4wj...|Applebee's Grill ...| Saturday| 47| 2019| 11.0|
5Q95IfYIbP6vXz4wj...|Applebee's Grill ...| Friday| 47| 2019| 11.0|
5Q95IfYIbP6vXz4wj...|Applebee's Grill ...| Thursday| 47| 2019| 11.0|
5Q95IfYIbP6vXz4wj...|Applebee's Grill ...|Wednesday| 47| 2019| 11.0|
5Q95IfYIbP6vXz4wj...|Applebee's Grill ...| Tuesday| 47| 2019| 11.0|
+--------------------+--------------------+---------+------------+-----------+---------+
only showing top 20 rows

In [0]:
df_insight2 = spark.sql("""
SELECT I.business_id, I.name, I.weekday, I.review_count, I.review_year, I.OpenHours, V.Breakfast, V.Brunch, V.Lunch, V.Dinner, V.LateNight
FROM insight1 AS I INNER JOIN vis AS V
ON I.business_id = V.business_id
""")
df_insight2.createOrReplaceTempView("insight2")
#df_insight2.write.mode("overwrite").saveAsTable("business_HoursReviewsInsight")
spark.sql("""
SELECT distinct(count(business_id)) from insight2""").show()

In [0]:
insight_check = spark.sql("""
SELECT distinct business_id, name,weekday,review_year, OpenHours, Breakfast, Brunch, Lunch, Dinner, LateNight, review_count
FROM insight2
""")
insight_meals = spark.sql("""
SELECT business_id, name,weekday,review_year, OpenHours, Breakfast, Brunch, Lunch, Dinner, LateNight, review_count
FROM insight2
group by business_id, review_year,name,weekday,review_year, OpenHours, Breakfast, Brunch, Lunch, Dinner, LateNight, review_count
ORDER BY business_id
""")
insight_meals.createOrReplaceTempView("mealsCheck")

###Insights on GoodForMeal Attribute

* There were **2,656** restaurants with a total of *31,221* reviews hat are good for breakfast only compared to **252** business with a total of *11,200* reviews who were good for everything but breakfast.
* There were **61,667** total restaurants that list False to every GoodForMeal attribute with *877,965* total reviews compared to **532** restuarants that list True for every GoodForMeal attribute with total review count of *38,059*.
* There were **15,191** total restaurants that list themselves as ONLY good for Lunch, totalling over *168,000* reviews.
* **1,320** restaurants list themselves as ONLY good for LateNight and totalled *19,498* reviews.
* 13,293 restaurants are ONLY good for Dinner with over *346,000* reviews.

### <a href="https://public.tableau.com/views/YelpDatareview/AttributeComparisoninRestaurantCountanditsReviews?:language=en&:display_count=y&:origin=viz_share_link">Visualization 3: Restaurant Reviews by their Corresponding Meal Type</a>

In [0]:
displayHTML("<div class='tableauPlaceholder' id='viz1607409756330' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ye&#47;YelpDatareview&#47;AttributeComparisoninRestaurantCountanditsReviews&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='YelpDatareview&#47;AttributeComparisoninRestaurantCountanditsReviews' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ye&#47;YelpDatareview&#47;AttributeComparisoninRestaurantCountanditsReviews&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1607409756330');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>")

#Restaurants in Metro Areas

####Cleaning up more data
We will first clean up the data by making sure the longitude and latitude matches with the State it is under. Some data may say they are from Arizona but the longtitude and latitude might say it is in Neveda. To fix this, we have to look at each state and the minimum and maximum values of both and say whether or not it is in the correct state. If it is, then we will give it a value of 0 meaning that the data is valid and a 1 meaning it is not.

In [0]:
df_good_data = spark.sql("""
SELECT business_id, name, state, latitude, longitude, review_count as reviews,
CASE  
  WHEN state = 'SC' AND latitude BETWEEN 34.89 AND 35.55 AND longitude BETWEEN -81.199640 AND -80.450430 THEN 0
  WHEN state = 'NC' AND latitude BETWEEN 34.89 AND 35.55 AND longitude BETWEEN -81.199640 AND -80.450430 THEN 0
  WHEN state = 'AZ' AND latitude BETWEEN 33.103471 AND 33.847847 AND longitude BETWEEN -112.542182 AND -111.600326 THEN 0
  WHEN state = 'AB' AND latitude BETWEEN 50.865169 AND 51.299943 AND longitude BETWEEN -114.379523 AND -113.714140 THEN 0
  WHEN state = 'IL' AND latitude BETWEEN 39.800029 AND 40.436864 AND longitude BETWEEN -88.579465 AND -87.831488 THEN 0
  WHEN state = 'OH' AND latitude BETWEEN 41.050003 AND 41.788514 AND longitude BETWEEN -82.24606 AND -81.05740 THEN 0
  WHEN state = 'ON' AND latitude BETWEEN 43.682467 AND 44.147909 AND longitude BETWEEN -79.91564 AND -78.90329 THEN 0
  WHEN state = 'PA' AND latitude BETWEEN 40.200117 AND 40.649427 AND longitude BETWEEN -80.293535 AND -79.650033 THEN 0
  WHEN state = 'QC' AND latitude BETWEEN 45.00967 AND 46.06701 AND longitude BETWEEN -73.45328 AND -72.80655 THEN 0
  WHEN state = 'NV' AND latitude BETWEEN 35.920416 AND 36.438249 AND longitude BETWEEN -115.286397 AND -114.800455 THEN 0
  WHEN state = 'WI' AND latitude BETWEEN 42.853384 AND 43.294981 AND longitude BETWEEN -89.749739 AND -89.102590 THEN 0
  ELSE 1
END AS valid 
FROM RestnBarsReview
""")
df_good_data.show(100,truncate=False)

+----------------------+--------------------------------+-----+-------------+---------------+-------+-----+
business_id |name |state|latitude |longitude |reviews|valid|
+----------------------+--------------------------------+-----+-------------+---------------+-------+-----+
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |133 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |165 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |165 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |132 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |78 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |120 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |12 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |41 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |159 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |184 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |1 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |157 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |139 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |171 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |159 |0 |
--DaPTJW3-tB1vP-PfdTEg|Sunnyside Grill |ON |43.6778069 |-79.4446742 |5 |1 |
--DaPTJW3-tB1vP-PfdTEg|Sunnyside Grill |ON |43.6778069 |-79.4446742 |3 |1 |
--DaPTJW3-tB1vP-PfdTEg|Sunnyside Grill |ON |43.6778069 |-79.4446742 |7 |1 |
--DaPTJW3-tB1vP-PfdTEg|Sunnyside Grill |ON |43.6778069 |-79.4446742 |8 |1 |
--DaPTJW3-tB1vP-PfdTEg|Sunnyside Grill |ON |43.6778069 |-79.4446742 |8 |1 |
--DaPTJW3-tB1vP-PfdTEg|Sunnyside Grill |ON |43.6778069 |-79.4446742 |3 |1 |
--DaPTJW3-tB1vP-PfdTEg|Sunnyside Grill |ON |43.6778069 |-79.4446742 |10 |1 |
--DaPTJW3-tB1vP-PfdTEg|Sunnyside Grill |ON |43.6778069 |-79.4446742 |7 |1 |
-0VImVWDeqewaA3th8fzIg|Bachelor Vegas |NV |36.0759923 |-115.1923434 |4 |0 |
-0VImVWDeqewaA3th8fzIg|Bachelor Vegas |NV |36.0759923 |-115.1923434 |4 |0 |
-0VImVWDeqewaA3th8fzIg|Bachelor Vegas |NV |36.0759923 |-115.1923434 |10 |0 |
-0VImVWDeqewaA3th8fzIg|Bachelor Vegas |NV |36.0759923 |-115.1923434 |13 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|24 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|16 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|13 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|23 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|14 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|17 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|17 |0 |
-0aIra_B6iALlfqAriBSYA|Tea2go |AZ |33.6754894 |-111.9649133 |7 |0 |
-0aIra_B6iALlfqAriBSYA|Tea2go |AZ |33.6754894 |-111.9649133 |1 |0 |
-0aIra_B6iALlfqAriBSYA|Tea2go |AZ |33.6754894 |-111.9649133 |11 |0 |
-0aIra_B6iALlfqAriBSYA|Tea2go |AZ |33.6754894 |-111.9649133 |14 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|2 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|1 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|1 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|6 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|4 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|1 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|2 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|4 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|2 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|

In [0]:
df_good_data.createOrReplaceTempView("good_data")

####Creating a Table for Tablaeu
After figuring out which data is good and which is bad, we will now only extract the good data to make sure we have only the correct data in order to create the table that will be used in the visualization software.

In [0]:
df_good_data1 = spark.sql("""
SELECT business_id, name, state, latitude, longitude, reviews, valid 
FROM good_data
WHERE valid = 0
""")
df_good_data1.show(100,truncate=False)                                  

+----------------------+--------------------------------+-----+-------------+---------------+-------+-----+
business_id |name |state|latitude |longitude |reviews|valid|
+----------------------+--------------------------------+-----+-------------+---------------+-------+-----+
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |133 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |165 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |165 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |132 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |78 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |120 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |12 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |41 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |159 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |184 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |1 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |157 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |139 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |171 |0 |
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |36.123183 |-115.16919 |159 |0 |
-0VImVWDeqewaA3th8fzIg|Bachelor Vegas |NV |36.0759923 |-115.1923434 |4 |0 |
-0VImVWDeqewaA3th8fzIg|Bachelor Vegas |NV |36.0759923 |-115.1923434 |4 |0 |
-0VImVWDeqewaA3th8fzIg|Bachelor Vegas |NV |36.0759923 |-115.1923434 |10 |0 |
-0VImVWDeqewaA3th8fzIg|Bachelor Vegas |NV |36.0759923 |-115.1923434 |13 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|24 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|16 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|13 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|23 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|14 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|17 |0 |
-0WegMt6Cy966qlDKhu6jA|Game Seven Grill |AZ |33.4455954039|-112.0684695343|17 |0 |
-0aIra_B6iALlfqAriBSYA|Tea2go |AZ |33.6754894 |-111.9649133 |7 |0 |
-0aIra_B6iALlfqAriBSYA|Tea2go |AZ |33.6754894 |-111.9649133 |1 |0 |
-0aIra_B6iALlfqAriBSYA|Tea2go |AZ |33.6754894 |-111.9649133 |11 |0 |
-0aIra_B6iALlfqAriBSYA|Tea2go |AZ |33.6754894 |-111.9649133 |14 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|2 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|1 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|1 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|6 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|4 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|1 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|2 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|4 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|2 |0 |
-2isRNVb6PDuBagELL5EBw|Taco Bell |AZ |33.5817130927|-112.1359007659|8 |0 |
-2wh_ZsD2n5xFYgzppcxNg|Bonga Korean Restaurant |AB |51.051027867 |-114.0619810671|7 |0 |
-2wh_ZsD2n5xFYgzppcxNg|Bonga Korean Restaurant |AB |51.051027867 |-114.0619810671|5 |0 |
-2wh_ZsD2n5xFYgzppcxNg|Bonga Korean Restaurant |AB |51.051027867 |-114.0619810671|9 |0 |
-2wh_ZsD2n5xFYgzppcxNg|Bonga Korean Restaurant |AB |51.051027867 |-114.0619810671|8 |0 |
-2wh_ZsD2n5xFYgzppcxNg|Bonga Korean Restaurant |AB |51.051027867 |-114.0619810671|5 |0 |
-33BHZKxCiREa1cfa672OQ|Triple Crown Bar & Grill |ON |43.7413837 |-79.3167491 |2 |0 |
-33BHZKxCiREa1cfa672OQ|Triple Crown Bar & Grill |ON |43.7413837 |-79.316

In [0]:
df_good_data1.write.mode("overwrite").saveAsTable("df_good_data_final")

##Number of Reviews Based on Size and Location of Metro Area
This visual shows the correlation between the number of reviews businesses gets based on their location. The darker the color, the more reviews each restaurant gets. The bigger the circle, the more businesses there are in the metro area. In two of the biggest Metro Areas, they have close to an average of 200 reviews per restaurant. This might be due to the fact that since it is a highly populated area, people are more likely to go out and spend money. However, in metro areas within Illinois and Wisconsin, they have less of a heaviliy populated area such as metro areas in Quebec, Alberta, Ontario but still have a similar average review counts as them. We can conclude from this visualization that the size or location of metro areas does not affect the amount of reviews each business gets.

In [0]:
displayHTML(showimage("/dbfs/FileStore/tables/MetroArea-1.PNG", 80))

## Working with Review Count and Alcohol Serving Styles
To see if serving alcohol has an effect on review count, we need all the businesses that serve alcohol based on the categories and alcohol attribute. We created a table already that contains this information called `RestnBars`. 

To start, we will filter out all the businesses will null values in the alcohol attribute.

In [0]:
df_alcohol = spark.sql("""
SELECT business_id, name, state,  attributes.Alcohol  
FROM restnbars
where attributes.Alcohol is not null
""")

print("total number of businesses with Alcohol:", df_alcohol.count())
df_alcohol.show() 

total number of businesses with Alcohol: 50838
+--------------------+--------------------+-----+----------------+
 business_id| name|state| Alcohol|
+--------------------+--------------------+-----+----------------+
pQeaRpvuhoEqudo3u...| The Empanadas House| IL| 'none'|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| ON| u'none'|
lu7vtrp_bE9PnxWfA...| Banzai Sushi| ON| u'none'|
9sRGfSVEfLhN_km60...| Apadana Restaurant| ON| u'full_bar'|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| AZ| u'none'|
fnZrZlqW1Z8iWgTVD...| Carl's Jr| NV| u'none'|
rVBPQdeayMYht4Uv_...|Gourmet Burger Co...| ON| u'none'|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| OH| u'none'|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| NV| u'none'|
Ga2Bt7xfqoggTypWD...| Amando's Bros| AZ| u'none'|
xFc50drSPxXkcLvX5...|Boomer's Sweet Ho...| AZ| u'none'|
tLpkSwdtqqoXwU0JA...| Wendy's| OH| u'none'|
Mmd5WDFq9hHcQ3uCl...| Irene's Tap Room| AZ|u'beer_and_wine'|
lK-wuiq8b1TuU7bfb...| Hingetown| OH| u'none'|
3BfGGIJn8lxvu1k3Z...| Théâtre Corona| QC| 'full_bar'|
-qjn24n8HYF6It9GQ...|Five Guys Burgers...| NC| u'none'|
ZkzutF0P_u0C0yTul...| Lelulos Pizzeria| PA| 'none'|
0QjROMVW9ACKjhSEf...| Mi Mi Restaurant| ON|u'beer_and_wine'|
OLciBerMDmD_WYzfY...|The Lounge at Sax...| NV| u'full_bar'|
7j0kor_fkeYhyEpXh...|Scramblers - Stro...| OH| u'none'|
+--------------------+--------------------+-----+----------------+
only showing top 20 rows

In [0]:
df_alcohol.createOrReplaceTempView("alcohol")

#### Cleaning Alcohol Attribute
Since the data we pulled from `RestnBars` included a 'u' in front of the alcohol attributes, we must clean the data so that it can be better understood.

In [0]:
df_alcohol_clean = spark.sql("""
select business_id,name,state, TRIM(LEADING 'u' FROM alcohol) alc
from alcohol
""")

print("Total number of businesses with Alcohol:", df_alcohol_clean.count())
df_alcohol_clean.show()

Total number of businesses with Alcohol: 50838
+--------------------+--------------------+-----+---------------+
 business_id| name|state| alc|
+--------------------+--------------------+-----+---------------+
pQeaRpvuhoEqudo3u...| The Empanadas House| IL| 'none'|
eBEfgOPG7pvFhb2wc...| Philthy Phillys| ON| 'none'|
lu7vtrp_bE9PnxWfA...| Banzai Sushi| ON| 'none'|
9sRGfSVEfLhN_km60...| Apadana Restaurant| ON| 'full_bar'|
vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| AZ| 'none'|
fnZrZlqW1Z8iWgTVD...| Carl's Jr| NV| 'none'|
rVBPQdeayMYht4Uv_...|Gourmet Burger Co...| ON| 'none'|
98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| OH| 'none'|
fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| NV| 'none'|
Ga2Bt7xfqoggTypWD...| Amando's Bros| AZ| 'none'|
xFc50drSPxXkcLvX5...|Boomer's Sweet Ho...| AZ| 'none'|
tLpkSwdtqqoXwU0JA...| Wendy's| OH| 'none'|
Mmd5WDFq9hHcQ3uCl...| Irene's Tap Room| AZ|'beer_and_wine'|
lK-wuiq8b1TuU7bfb...| Hingetown| OH| 'none'|
3BfGGIJn8lxvu1k3Z...| Théâtre Corona| QC| 'full_bar'|
-qjn24n8HYF6It9GQ...|Five Guys Burgers...| NC| 'none'|
ZkzutF0P_u0C0yTul...| Lelulos Pizzeria| PA| 'none'|
0QjROMVW9ACKjhSEf...| Mi Mi Restaurant| ON|'beer_and_wine'|
OLciBerMDmD_WYzfY...|The Lounge at Sax...| NV| 'full_bar'|
7j0kor_fkeYhyEpXh...|Scramblers - Stro...| OH| 'none'|
+--------------------+--------------------+-----+---------------+
only showing top 20 rows

In [0]:
df_alcohol_clean.createOrReplaceTempView("alcohol_clean")

After cleaning the Alcohol attribute, we need to identify the exact number of businesses that serve alcohol. To do this, we first revise the alcohol attribute names to a cleaner form so that we can then group it and find the total count.

In [0]:
df_alcohol_revised = spark.sql("""
select CASE WHEN alc LIKE '%none%' or alc = 'None' THEN 'None'
WHEN alc LIKE '%beer_and_wine%' THEN 'Beer and Wine'
WHEN alc LIKE '%full_bar%' THEN 'Full Bar'
ELSE 'Unknown'
END AS alcohol_serving, business_id,name,state
from alcohol_clean
""")
print("Total number of businesses with Alcohol:", df_alcohol_revised.count())
df_alcohol_revised.show(60) 

Total number of businesses with Alcohol: 50838
+---------------+--------------------+--------------------+-----+
alcohol_serving| business_id| name|state|
+---------------+--------------------+--------------------+-----+
 None|pQeaRpvuhoEqudo3u...| The Empanadas House| IL|
 None|eBEfgOPG7pvFhb2wc...| Philthy Phillys| ON|
 None|lu7vtrp_bE9PnxWfA...| Banzai Sushi| ON|
 Full Bar|9sRGfSVEfLhN_km60...| Apadana Restaurant| ON|
 None|vjTVxnsQEZ34XjYNS...| Wetzel's Pretzels| AZ|
 None|fnZrZlqW1Z8iWgTVD...| Carl's Jr| NV|
 None|rVBPQdeayMYht4Uv_...|Gourmet Burger Co...| ON|
 None|98hyK2QEUeI8v2y0A...|Pho Lee's Vietnam...| OH|
 None|fhNf_sg-XzZ3e7HEV...|Meat Chix And Wie...| NV|
 None|Ga2Bt7xfqoggTypWD...| Amando's Bros| AZ|
 None|xFc50drSPxXkcLvX5...|Boomer's Sweet Ho...| AZ|
 None|tLpkSwdtqqoXwU0JA...| Wendy's| OH|
 Beer and Wine|Mmd5WDFq9hHcQ3uCl...| Irene's Tap Room| AZ|
 None|lK-wuiq8b1TuU7bfb...| Hingetown| OH|
 Full Bar|3BfGGIJn8lxvu1k3Z...| Théâtre Corona| QC|
 None|-qjn24n8HYF6It9GQ...|Five Guys Burgers...| NC|
 None|ZkzutF0P_u0C0yTul...| Lelulos Pizzeria| PA|
 Beer and Wine|0QjROMVW9ACKjhSEf...| Mi Mi Restaurant| ON|
 Full Bar|OLciBerMDmD_WYzfY...|The Lounge at Sax...| NV|
 None|7j0kor_fkeYhyEpXh...|Scramblers - Stro...| OH|
 None|j9bWpCRwpDVfwVT_V...| Papaya Thai| AZ|
 None|6GHwgKNlvfIMUpFax...|L'Artisan Piadineria| QC|
 Full Bar|0Y5Kzo8PWHTjk0tlf...| Caballero| QC|
 None|JcsZvx-4yovFgCXOf...| Sam's Deli Mart| AZ|
 None|MTx-Zdl_KcU_z9G83...| Taco Bell| NV|
 Full Bar|a6d7UcYnRvbr4t-TH...| The Pub| OH|
 Beer and Wine|UFU8ONTkzEkcOk61X...| Nickels| QC|
 None|W7hCuNdn2gzehta6e...| Pete's Fish & Chips| AZ|
 None|8nP8ghEpT6WFcM6tf...| Sugar Bowl| AZ|
 None|8k62wYhDVq1-652Yb...| Tim Hortons| ON|
 Beer and Wine|8Hvp1tYKiQbBgGIwk...|Tony's Family Res...| OH|
 Full Bar|kQknjbOvtPmS3NVm-...|Hotel Charlotte R...| NC|
 None|0y6alZmSLnPzmG5_k...| J J's Pizza| OH|
 Full Bar|OGVHlFHSXjHuioOvm...|The Jackman Bar &...| PA|
 None|Q_dh08clYUPj13GmC...|Kulinarya Express...| NV|
 None|3C0bnFhjkgYP9mWOR...| Chili Man| NC|
 Full Bar|t749-3jKKDyO8QH3V...| Chao Phraya| QC|
 Full Bar|ydE2SjvSd9_NMoEBA...| Chili's Grill & Bar| AZ|
 None|0DnQh8SE8BSnvJltG...| Chick-N-Joy| ON|
 None|5X7TQsA3iDbBc8yy2...| Blimpie| AZ|
 Beer and Wine|e1tqT4QNOUqDiOvma...|Chef Alyssa's Kit...| NC|
 Full Bar|VBVqHnViRLSYAqP7v...| Jalapeño Inferno| AZ|
 None|Yr_w9lakJrKMyEG_h...|Fat Moe's Pizza &...| NV|
 Full Bar|cicPsia8Wj-DNRkmL...|The Keg Steakhous...| ON|
 Full Bar|2aWV8Rx4TwtB0x2pf...|Restaurant Housto...| QC|
 None|0nqQIM2PvOcTX6v0L...| Fearless Meat| ON|
 None|rzd_2GJpkvHPejEwJ...| Somer's Restaurant| OH|
 Beer and Wine|PlP0c_u3g-3LiWHpb...| Lee Cafe| SC|
 Beer and Wine|o9auqDMXSNvE_jNQb...| Sushi Club K'z| AB|
 Full Bar|xVXyrTWbG8U3szze-...|St. Louis Bar & G...| ON|
 None|YHWjW9GLcuNtnh_xd...| Pizza Fresh| AZ|
 None|8k8cMmMcuj5h85mML...| RayNathan's| NC|
 None|GvCblda5MdYBqNFBU...|Barb's Country Ju...| PA|
 None|1aVqiz43klXaFJUUx...| Dutch Bros Coffee| AZ|
 None|Yy_SB9r7VQTlIVv_M...| Cari Mela| QC|
 Beer and Wine|r3jpH6ZuCQY3o1yBu...| Master Mix HotPot| ON|
 Full Bar|yXgmi4C_GQe-Vn47a...| Pompette| QC|
 None|AN0bWhisCf6LN9eHZ...|Los Olivos Ristor...| NV|
 None|goar7zF4G0LdsQ1Y4...|Pittsburgh Barbec...| PA|
 None|AtD6B83S4Mbmq0t7i...| Veggie House| NV|
+---------------+--------------------+--------------------+-----+
only showing top 60 rows

In [0]:
df_alcohol_revised.createOrReplaceTempView("alcohol_revised")

In [0]:
df_alcohol_revised_summary = df_alcohol_revised.select("business_id", "alcohol_serving").groupBy("alcohol_serving").count()

df_alcohol_revised_summary.show()

+---------------+-----+
alcohol_serving|count|
+---------------+-----+
 None|22764|
 Beer and Wine| 6984|
 Full Bar|21090|
+---------------+-----+

####Visualizing Alcohol and Total Number of Businesses
Here, we see the the total number of restaurants that serve alcohol against those that do not. The graph compares the total number of businesses with how many of them fall under the specific alcohol attribute.

In [0]:
display(df_alcohol_revised_summary.orderBy("alcohol_serving"))

alcohol_serving,count
Beer and Wine,6984
Full Bar,21090
None,22764


#### Joining Review Count and Alcohol

Now that we have cleaned up the alcohol attribute, we can use a left join to join together `alcohol_revised` with `reviewcount` to link the businesses that serve alcohol with the total number of reviews they receive.

Since this information is important to our visualization in Tableau, we will also save it as `alcohol_reviews` table

In [0]:
df_restaurant_alcohol_reviews = spark.sql("""
SELECT a.alcohol_serving, a.business_id, a.name, r.reviews
FROM alcohol_revised as a LEFT JOIN reviewcount as r 
ON a.business_id = r.business_id
""")

print("Total number of businesses with Alcohol:", df_restaurant_alcohol_reviews.count())
df_restaurant_alcohol_reviews.show()

Total number of businesses with Alcohol: 50838
+---------------+--------------------+--------------------+-------+
alcohol_serving| business_id| name|reviews|
+---------------+--------------------+--------------------+-------+
 None|DloBjCZHlz0lzsrQJ...| The Lit Grille| 2|
 Full Bar|x1fx7C_tc064cFATj...|The Lodge at Gran...| 11|
 Full Bar|ATeOjZzE3VOI-RW7F...| One Red Door| 10|
 Full Bar|VHsNB3pdGVcRgs6C3...| Dublin O'Neil's| 8|
 Full Bar|RMjCnixEY5i12Ciqn...| Four Star Sushi| 8|
 Beer and Wine|BQMVcJQV70gUQjwi7...|Gustoso Authentic...| 2|
 None|M9N2eaO8PkT7MHGLL...| Panda Express| 2|
 Full Bar|juNKWhkMynLaIHK4Y...| Eton House Tavern| 9|
 Full Bar|WXd7r6Yvjxhyo1L6R...|Mario's South Sid...| 13|
 Beer and Wine|_Wyo52ijeq3QgcmkI...|Lemongrass Restau...| 11|
 Beer and Wine|YYWkJwfO1HApuHiW-...| Stevie's Pizza| 7|
 Full Bar|orvDK3kwkIZ7iS0UT...|Erie Waters Sport...| 3|
 None|m3ENrv6bKW8h44bMy...| Subway Restaurants| 7|
 None|c-N5b3LFflVwPwqwi...| Palm Lane| 3|
 None|EgNqQM_pzECJVJk1h...| Vegas Family Vision| 4|
 None|4x8W4quFhhuTZlsoU...| McDonald's| 10|
 Full Bar|encgrnMKe8Z37FMlM...| Ideal Bar| 7|
 None|fdBB7vuO3NnMps5DZ...|Blaze Fast-Fire'd...| 2|
 Full Bar|ipFreSFhjClfNETuM...| Doogan's of Aurora| 8|
 None|7u8PwKlfk7rKIPfe7...| Rosati's Pizza| 5|
+---------------+--------------------+--------------------+-------+
only showing top 20 rows

In [0]:
df_restaurant_alcohol_reviews.write.mode("overwrite").saveAsTable("alcohol_reviews")

### Visualizing Alcohol and Review Count

To begin visualizing how an alcohol service style can affect review count, we use the dataframe we just created, `df_restaurant_alcohol_reviews` and we group the results based on the alcohol serving style. This will give us a total count of the number of businesses that serve alcohol as a full bar, only beer and wine, or do not include any information regarding that attribute.

In [0]:
df_restaurant_alcohol_reviews_summary = df_restaurant_alcohol_reviews.select("alcohol_serving", "reviews").groupBy("alcohol_serving").count()
df_restaurant_alcohol_reviews_summary.show()

+---------------+-----+
alcohol_serving|count|
+---------------+-----+
 None|22764|
 Beer and Wine| 6984|
 Full Bar|21090|
+---------------+-----+

Based on the summary chart that we generated above, we see that restaurants that do not offer alcohol have a higher review count than those that do. But, if we combine the number of restaurants that serve either Beer and Wine or offer a Full Bar, there are more reviews in total.

In [0]:
display(df_restaurant_alcohol_reviews_summary.orderBy("alcohol_serving"))

alcohol_serving,count
Beer and Wine,6984
Full Bar,21090
None,22764


## Does serving alcohol affect review count? 
Based on the visualizations we generated from the data, we can see that businesses that offer alcohol generally receive more reviews than those that do not. Of the restaurants that do serve alcohol, offering a Full Bar leads to the most reviews.

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/alcoholxreviews-3.png"))

d
#Conclusion
###What does Yelp want to know?

1.Do the hours that restaurants and bars are open each week relate to the number of reviews they get per year?
  
2.What role does the size or location of the metro area play (do smaller cities have fewer reviews per restaurant)?

3.Does serving alcohol matter?
  
4.Does being open for certain meals matter?
   

####Our Findings

1. The size or the location of the metro area plays a huge role when it comes to the review count. As shown in the visualizations it indicates that the bigger the metro area the more reviews the restaurants will receive from their customers. The reason is transparent as more customers will enagage in a bigger metro area and will eventually dine in the restaurants which are in the that specific metro areas. On the other hand, restaurants that are in smaller metro areas will recieve less reviews. 

2. We found out when looking at the number of reviews they obtain based on the number of hours they are open, there is a normal distrubution. Restaurants which are open 60 to 80 hours a week have the most reviews in every year in the data with a minimum of 0 hours and a maximum of 168 hours. We can conclude that restaurants that are open for 60-80 hours will most likely gain a higher review count. Restaurants that are open for 24 hours do  receive a good amount of reviews but not much compared to ones that are selective with their hours. 

3. We also found that serving alcohol has an huge impact on the review count for the restaurants in the area. For example, restaurants which are serving a full bar have more reviews than compared to businesses which only have beer and wine section or none at all. 

4. Based on the GoodForMeal Attributes our findings conclude that restaurants which identify themselves as good for lunch or dinner have highest review count than any other GoodForMeal factor. 


####Limitations 
1.Not all restaurants filled out some of the attributes such as the "Good for Meal". Out of all the restaurants and bars in our dataset, about 68,000 data entries, only half of them had filled out the "Good For Meal". 

2.Within the data, some restaurant/bars did not say whether or not they had either beer/wine or a full bar. In order to not skew the data and put a restaurant/bar into one of those two categories, we deselected the rows that gave a null value. We are limited to what we know because of the null values.